In [0]:
%pip install requests_oauthlib

In [0]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests.auth import HTTPBasicAuth
from requests_oauthlib import OAuth2Session
import json
from pyspark.sql import functions as f

In [0]:
def generate_headers(client_id,client_secret,token_endpoint):
    # Create a BackendApplicationClient object
    client = BackendApplicationClient(client_id=client_id)

    # Create an OAuth2Session object
    oauth = OAuth2Session(client=client)

    # Get the access token
    token = oauth.fetch_token(
        token_url= token_endpoint,
        auth=HTTPBasicAuth(client_id, client_secret),
        scope= "e451.api.access"
    ) 
    # Create headers
    headers = {
        'Authorization': 'Bearer ' + token['access_token']
    }
    return headers

In [0]:
def declare_dev_credentials():
  client_id = '0oaco04jd1oGa4rNW1d7'
  client_secret = 'e6tOus3jPGoP2S96zbX9IDHID1VsPK_YC8i77MUgzaNi6CILXs_ux5ia8FYIDolO'
  token_endpoint = 'https://login-dev.8451.com/oauth2/aus4a5n1x8HRjB7LS1d7/v1/token'
  campaign_get_endpoint = 'https://map-campaign-service.aks-ur-' + 'dev' + '-internal.8451.cloud/v0/campaign?id='
  campaign_patch_endpoint= 'https://map-campaign-service.aks-ur-' + 'dev' + '-internal.8451.cloud/v0/campaign/' 
  return campaign_get_endpoint,campaign_patch_endpoint,token_endpoint,client_id,client_secret

In [0]:
def declare_prd_credentials():
  client_id= '0oab79tqp4TYivqwr697'
  client_secret = 'gd-cDiEKQ9FKigY8-mnsgNl139Cmwhoh2vqE2p1_5WcPTYVSPoTV0pKG13Qzhelb'
  token_endpoint = 'https://login.8451.com/oauth2/aus2rfaog9lmi37qz697/v1/token'
  campaign_get_endpoint = 'https://map-campaign-service.aks-ur-' + 'prd' + '-internal.8451.cloud/v0/campaign?id='
  campaign_patch_endpoint= 'https://map-campaign-service.aks-ur-' + 'prd' + '-internal.8451.cloud/v0/campaign/'
  return campaign_get_endpoint,campaign_patch_endpoint,token_endpoint,client_id,client_secret

In [0]:
## reading csv file
def read_campaign_patch_file(filename):
  return spark.read.option('header','True').csv(f'/FileStore/Users/s197818@8451.com/testing/{filename}')

In [0]:
def create_json_variables(df):
  path_list = [row.path for row in df.select('path').collect()]
  value_list = [row.value for row in df.select('value').collect()]
  op_list = [row.op for row in df.select('op').collect()]
  campaign_ids = [row.campaign_id for row in df.select('campaign_id').collect()]
  return path_list, value_list, op_list, campaign_ids

In [0]:
def create_payload(val,op,path):
  ## catching the ValueError exception
  try:
    val = int(val)
  except:
    print("Value is NOT AN INTEGER!!!")
  payload = [{
    'op': op, 
    'path': path,
    'value': val
    }]
  return payload